In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [ ]:
kp_cluster_df = pd.read_csv('../K-Prototypes/results/set3_k10.csv', dtype={
    'uid': str,
    'Gender_Code': str, 
    'ZIP_Code': str, 
    'AD_TAG': str, 
    'INV_time_mode': str, 
    'APP_time_mode': str, 
    'TXN_time_mode': str, 
    'TXN_zip_code': str
    })

In [ ]:
kp_cluster_df.columns

# Categorical Columns LabelEncoder

In [2]:
cat_columns = ['Gender_Code', 'ZIP_Code',
       'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode',
       'TXN_zip_code']

In [15]:
from sklearn.preprocessing import LabelEncoder

def LabelEncoding(df, columns):
    le = LabelEncoder()
    for column in columns:
        df[column] = le.fit_transform(df[column])
    
    return df

In [67]:
LabelEncoding(kp_cluster_df, cat_columns)

kp_cluster_df.sort_values(by=['cluster_labels'], inplace=True)
kp_cluster_df

,INV_total_amount,INV_count,INV_time_avg_period,APP_participate_count,APP_bnnr_per,APP_game_per,APP_time_avg_period,TXN_total_count,TXN_earn_count_per,TXN_redeem_count_per,...,TXN_industry_per_food,TXN_time_avg_period,Gender_Code,ZIP_Code,AD_TAG,INV_time_mode,APP_time_mode,TXN_time_mode,TXN_zip_code,cluster_labels
23218,-0.496787,-0.902011,0.056025,-0.343628,-0.536683,-0.605184,-0.373737,-0.593437,0.654441,-0.551233,...,-0.010644,0.033392,1,5,5,14,6,23,2053,0
193549,0.051809,0.506650,-0.332592,-0.343628,-0.536683,-0.605184,-0.492807,-0.530558,0.654441,-0.551233,...,-0.010644,1.859580,1,39,10,8,10,4514,2053,0
74712,-0.288990,-0.231220,-0.295371,-0.127739,-0.536683,1.973945,-0.481781,-0.216161,0.654441,-0.551233,...,-0.010644,-0.393644,2,315,9,18,3,993,1522,0
170073,0.009123,0.204794,-0.321916,-0.343628,-0.536683,-0.605184,-0.492740,0.349754,0.470819,-0.187892,...,-0.010644,-0.621851,1,122,4,3916,13,23,4,0
219082,-0.430927,-0.633695,-0.183743,-0.327021,2.302794,-0.605184,-0.385455,-0.593437,0.654441,-0.551233,...,-0.010644,-0.386531,2,119,10,21,10,23,2053,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291387,0.144129,0.137715,-0.317617,-0.343628,-0.536683,-0.605184,-0.484739,-0.153281,-0.508498,1.749925,...,-0.010644,-0.555946,1,26,2,4544,13,20,22,9
291386,-0.101263,-0.432457,-0.260890,-0.277201,0.883056,0.684380,-0.487922,-0.341919,0.042368,0.659903,...,-0.010644,-0.678876,1,5,5,8,23,23,399,9
291385,0.401634,-0.164141,-0.293085,-0.277201,0.173186,1.329163,-0.472782,0.349754,0.287197,0.175449,...,-0.010644,-0.705391,2,66,3,4993,16,23,58,9
172263,0.298451,-0.180911,-0.293761,-0.293807,2.302794,-0.605184,-0.464498,0.538392,0.011764,0.720460,...,-0.010644,-0.689403,1,85,1,23,9,2606,76,9


# Cluster Labels Value Counts

In [18]:
kp_cluster_df['cluster_labels'].value_counts()

5    79195
9    69618
2    35599
8    34877
3    33673
0    23246
7    20384
1     6177
4     5872
6      250
Name: cluster_labels, dtype: int64

# DecisionTreeClassifier

In [60]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
# from sklearn.preprocessing import StandardScaler
import graphviz
from sklearn import metrics

def buildtree(df, feature):
    X = df[feature].values
    y = df[['label']].values
    clf = DecisionTreeClassifier(max_leaf_nodes = 7, min_samples_split = 30000, class_weight={1:0.4,2:0.6,0:0.8,3:1,4:2,8:2,5:3})
    clf = clf.fit(X, y)

    # dot_data = export_graphviz(clf)
    dot_data = export_graphviz(clf, 
                        feature_names = feature,
                        class_names = df['label'].unique().astype(str),
                        rounded = True,
                        filled = True,
                        )
    graph = graphviz.Source(dot_data)
    graph.view()
    
    dictionary = dict(zip(feature, clf.feature_importances_))
    print('feature_importances', dictionary)

    test_y_predicted = clf.predict(X)

    # accuracy
    accuracy = metrics.accuracy_score(y, test_y_predicted)
    print('accuracy', accuracy)

# K-Prototypes

In [147]:
feature = kp_cluster_df.columns[:-2]

buildtree(kp_cluster_df, feature)

feature_importances {'INV_total_amount': 0.0, 'INV_count': 0.0, 'INV_time_avg_period': 0.0, 'APP_participate_count': 0.13524287441898608, 'APP_bnnr_per': 0.0, 'APP_game_per': 0.0, 'APP_time_avg_period': 0.0, 'TXN_total_count': 0.0, 'TXN_earn_count_per': 0.0, 'TXN_redeem_count_per': 0.0, 'TXN_total_point': 0.0, 'TXN_earn_point_per': 0.20960901321850547, 'TXN_redeem_point_per': 0.19561396859579114, 'TXN_industry_count': 0.0, 'TXN_industry_per_retail': 0.17387900990704383, 'TXN_industry_per_other': 0.040209844871197574, 'TXN_industry_per_dingding': 0.041610153583166505, 'TXN_industry_per_financial': 0.052053223359229996, 'TXN_industry_per_digit': 0.15178191204607938, 'TXN_industry_per_transportation': 0.0, 'TXN_industry_per_leisure': 0.0, 'TXN_industry_per_health': 0.0, 'TXN_industry_per_professional': 0.0, 'TXN_industry_per_food': 0.0, 'TXN_time_avg_period': 0.0, 'Gender_Code': 0.0, 'ZIP_Code': 0.0, 'AD_TAG': 0.0, 'INV_time_mode': 0.0, 'APP_time_mode': 0.0, 'TXN_time_mode': 0.0}
accuracy

# Return STD and Mean of Numeric Data

In [62]:
original_data = pd.read_csv('../DATA/HappyGo_NOV_DEC_0509_FV.csv', dtype={
                            'uid': str,
                            'Gender_Code': str, 
                            'ZIP_Code': str, 
                            'AD_TAG': str, 
                            'INV_time_mode': str, 
                            'APP_time_mode': str, 
                            'TXN_time_mode': str, 
                            'TXN_zip_code': str
                            })
original_data = original_data[['INV_total_amount', 'INV_count', 'INV_time_avg_period',
       'APP_participate_count', 'APP_bnnr_per', 'APP_game_per',
       'APP_time_avg_period', 'TXN_total_count', 'TXN_earn_count_per',
       'TXN_redeem_count_per', 'TXN_total_point', 'TXN_earn_point_per',
       'TXN_redeem_point_per', 'TXN_industry_count', 'TXN_industry_per_retail',
       'TXN_industry_per_other', 'TXN_industry_per_dingding',
       'TXN_industry_per_financial', 'TXN_industry_per_digit',
       'TXN_industry_per_transportation', 'TXN_industry_per_leisure',
       'TXN_industry_per_health', 'TXN_industry_per_professional',
       'TXN_industry_per_food', 'TXN_time_avg_period', 'Gender_Code',
       'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode',
       'TXN_zip_code']]

original_mean = dict(zip(original_data.select_dtypes(include='float'), original_data.select_dtypes(include='float').mean()))
original_std = dict(zip(original_data.select_dtypes(include='float'), original_data.select_dtypes(include='float').std()))

In [63]:
def inverse_standarlization(z, mean, std):
    x = z * std + mean
    return x

In [115]:
# kp set3&k=10
print(
    'split 1:', inverse_standarlization(0.146, original_mean['TXN_earn_point_per'], original_std['TXN_earn_point_per']), '\n'
    'split 2:', inverse_standarlization(-0.315, original_mean['TXN_redeem_point_per'], original_std['TXN_redeem_point_per']), '\n'
    'split 3:', inverse_standarlization(-0.052, original_mean['TXN_industry_per_retail'], original_std['TXN_industry_per_retail']), '\n'
    'split 4:', inverse_standarlization(0.445, original_mean['APP_participate_count'], original_std['APP_participate_count']), '\n'
    'split 5:', inverse_standarlization(1.282, original_mean['TXN_industry_per_digit'], original_std['TXN_industry_per_digit']), '\n'
    'split 6:', inverse_standarlization(1.7, original_mean['TXN_industry_per_digit'], original_std['TXN_industry_per_digit']), '\n'
    'split 7:', inverse_standarlization(0.83, original_mean['TXN_industry_per_dingding'], original_std['TXN_industry_per_dingding']), '\n'
    'split 8:', inverse_standarlization(1.603, original_mean['TXN_industry_per_financial'], original_std['TXN_industry_per_financial']), '\n'
    'split 9:', inverse_standarlization(3.264, original_mean['TXN_industry_per_other'], original_std['TXN_industry_per_other']), '\n'
    )

split 1: 0.6650095121842882 
split 2: 0.16984715202179282 
split 3: 0.3948745156528075 
split 4: 47.48822055883487 
split 5: 0.46504952366208296 
split 6: 0.5750127410482362 
split 7: 0.5050530813907729 
split 8: 0.29506289683560816 
split 9: 0.32501907112222755 



In [71]:
# birch set3&k=21
print(
    'split 1:', inverse_standarlization(-2.345, original_mean['TXN_earn_count_per'], original_std['TXN_earn_count_per']), '\n'
    'split 2:', inverse_standarlization(0.598, original_mean['APP_time_avg_period'], original_std['APP_time_avg_period']), '\n'
    'split 3:', inverse_standarlization(0.067, original_mean['TXN_total_count'], original_std['TXN_total_count']), '\n'
    'split 4:', inverse_standarlization(0.115, original_mean['APP_time_avg_period'], original_std['APP_time_avg_period']), '\n'
    'split 5:', inverse_standarlization(0.57, original_mean['TXN_total_count'], original_std['TXN_total_count']), '\n'
    'split 6:', inverse_standarlization(-0.13, original_mean['TXN_time_avg_period'], original_std['TXN_time_avg_period']), '\n'
    )

split 1: 0.01990692119702253 
split 2: 2097595.674489021 
split 3: 11.503234582319756 
split 4: 1168797.944966062 
split 5: 19.502685752472086 
split 6: 1115243.9514100389 



# BIRCH best result

In [13]:
birch_cluster_df = pd.read_csv('../BIRCH/birch_best_result.csv', dtype={
    'uid': str,
    'Gender_Code': str, 
    'ZIP_Code': str, 
    'AD_TAG': str, 
    'INV_time_mode': str, 
    'APP_time_mode': str, 
    'TXN_time_mode': str, 
    'TXN_zip_code': str
    })
birch_cluster_df.sort_values(by=['label'], inplace=True)
birch_cluster_df

,UID,INV_total_amount,INV_avg_amount,INV_count,INV_seller_count,INV_industry_count,INV_industry_per_catering,INV_industry_per_entertainment,INV_industry_per_financial_insurance,INV_industry_per_health_and_beauty,...,TXN_industry_per_health,TXN_industry_per_professional,TXN_industry_per_food,TXN_time_mode,TXN_zip_code,TXN_time_avg_period,AD_TAG,Gender_Code,ZIP_Code,label
295548,9156065,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,['00' '12'],744,293342.0,P05,M,744,0
42559,1567328,63651.0,636.510000,100.0,34.0,5.0,0.110000,0.000000,0.0,0.0,...,0.0,0.0,0.0,23,106,285681.0,P04,F,420,0
81685,6300480,216787.0,1885.104348,115.0,32.0,7.0,0.060870,0.208696,0.0,0.0,...,0.0,0.0,0.0,12,111,156765.0,P01,F,252,0
42563,1249681,6883.0,229.433333,30.0,12.0,5.0,0.066667,0.000000,0.0,0.0,...,0.0,0.0,0.0,00,114,297016.0,P10,F,220,0
173421,1262870,13135.0,505.192308,26.0,12.0,5.0,0.269231,0.000000,0.0,0.0,...,0.0,0.0,0.0,23,106,431421.0,P03,M,236,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109925,63965,260199.0,1161.602679,224.0,38.0,7.0,0.022321,0.022321,0.0,0.0,...,0.0,0.0,0.0,12,220,175417.0,P03,M,237,16
117646,8883154,42799.0,555.831169,77.0,6.0,5.0,0.012987,0.012987,0.0,0.0,...,0.0,0.0,0.0,12,432,9327.0,P05,F,401,17
130558,5775543,158176.0,648.262295,244.0,46.0,7.0,0.061475,0.020492,0.0,0.0,...,0.0,0.0,0.0,12,106,6778.0,P02,F,112,18
165762,3629062,944394.0,5902.462500,160.0,60.0,7.0,0.443750,0.006250,0.0,0.0,...,0.0,0.0,0.0,15,234,2748.0,P01,M,220,19


In [68]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std_feature = std.fit_transform(birch_cluster_df[['INV_total_amount', 'INV_count', 'INV_time_avg_period', 'APP_participate_count', 'APP_bnnr_per', 'APP_game_per', 'APP_time_avg_period', 'TXN_total_count', 'TXN_earn_count_per', 'TXN_redeem_count_per', 
                 'TXN_total_point', 'TXN_earn_point_per', 'TXN_redeem_point_per', 'TXN_industry_count', 'TXN_industry_per_retail', 'TXN_industry_per_other', 'TXN_industry_per_dingding', 'TXN_industry_per_financial', 
                 'TXN_industry_per_digit', 'TXN_industry_per_transportation', 'TXN_industry_per_leisure', 'TXN_industry_per_health', 'TXN_industry_per_professional', 'TXN_industry_per_food', 'TXN_time_avg_period']])
birch_cluster_df[['INV_total_amount', 'INV_count', 'INV_time_avg_period', 'APP_participate_count', 'APP_bnnr_per', 'APP_game_per', 'APP_time_avg_period', 'TXN_total_count', 'TXN_earn_count_per', 'TXN_redeem_count_per', 
                 'TXN_total_point', 'TXN_earn_point_per', 'TXN_redeem_point_per', 'TXN_industry_count', 'TXN_industry_per_retail', 'TXN_industry_per_other', 'TXN_industry_per_dingding', 'TXN_industry_per_financial', 
                 'TXN_industry_per_digit', 'TXN_industry_per_transportation', 'TXN_industry_per_leisure', 'TXN_industry_per_health', 'TXN_industry_per_professional', 'TXN_industry_per_food', 'TXN_time_avg_period']] = std_feature

birch_cluster_df

,UID,INV_total_amount,INV_avg_amount,INV_count,INV_seller_count,INV_industry_count,INV_industry_per_catering,INV_industry_per_entertainment,INV_industry_per_financial_insurance,INV_industry_per_health_and_beauty,...,TXN_industry_per_health,TXN_industry_per_professional,TXN_industry_per_food,TXN_time_mode,TXN_zip_code,TXN_time_avg_period,AD_TAG,Gender_Code,ZIP_Code,label
295548,9156065,-0.519434,0.000000,-1.019399,0.0,0.0,0.000000,0.000000,0.0,0.0,...,-0.033579,-0.054966,-0.010644,['00' '12'],744,-0.632675,P05,M,744,0
42559,1567328,0.439635,636.510000,0.657578,34.0,5.0,0.110000,0.000000,0.0,0.0,...,-0.033579,-0.054966,-0.010644,23,106,-0.637360,P04,F,420,0
81685,6300480,2.747029,1885.104348,0.909124,32.0,7.0,0.060870,0.208696,0.0,0.0,...,-0.033579,-0.054966,-0.010644,12,111,-0.716205,P01,F,252,0
42563,1249681,-0.415724,229.433333,-0.516306,12.0,5.0,0.066667,0.000000,0.0,0.0,...,-0.033579,-0.054966,-0.010644,00,114,-0.630428,P10,F,220,0
173421,1262870,-0.321521,505.192308,-0.583385,12.0,5.0,0.269231,0.000000,0.0,0.0,...,-0.033579,-0.054966,-0.010644,23,106,-0.548226,P03,M,236,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109925,63965,3.401144,1161.602679,2.737029,38.0,7.0,0.022321,0.022321,0.0,0.0,...,-0.033579,-0.054966,-0.010644,12,220,-0.704798,P03,M,237,16
117646,8883154,0.125445,555.831169,0.271873,6.0,5.0,0.012987,0.012987,0.0,0.0,...,-0.033579,-0.054966,-0.010644,12,432,-0.806378,P05,F,401,17
130558,5775543,1.863901,648.262295,3.072424,46.0,7.0,0.061475,0.020492,0.0,0.0,...,-0.033579,-0.054966,-0.010644,12,106,-0.807937,P02,F,112,18
165762,3629062,13.710329,5902.462500,1.663764,60.0,7.0,0.443750,0.006250,0.0,0.0,...,-0.033579,-0.054966,-0.010644,15,234,-0.810402,P01,M,220,19


In [17]:
birch_cluster_df['label'].value_counts()

1     94744
2     68962
0     51688
3     29660
4     22551
8     20677
5     13761
7      5504
6      1062
9       182
10       56
12       16
15       13
13        7
11        2
14        1
16        1
17        1
18        1
19        1
20        1
Name: label, dtype: int64

In [69]:
feature = birch_cluster_df[['INV_total_amount', 'INV_count', 'INV_time_avg_period', 'APP_participate_count', 'APP_bnnr_per', 'APP_game_per', 'APP_time_avg_period', 'TXN_total_count', 'TXN_earn_count_per', 'TXN_redeem_count_per', 
                 'TXN_total_point', 'TXN_earn_point_per', 'TXN_redeem_point_per', 'TXN_industry_count', 'TXN_industry_per_retail', 'TXN_industry_per_other', 'TXN_industry_per_dingding', 'TXN_industry_per_financial', 
                 'TXN_industry_per_digit', 'TXN_industry_per_transportation', 'TXN_industry_per_leisure', 'TXN_industry_per_health', 'TXN_industry_per_professional', 'TXN_industry_per_food', 'TXN_time_avg_period']]

buildtree(birch_cluster_df, feature.columns)

feature_importances {'INV_total_amount': 0.0, 'INV_count': 0.0, 'INV_time_avg_period': 0.0, 'APP_participate_count': 0.0, 'APP_bnnr_per': 0.0, 'APP_game_per': 0.0, 'APP_time_avg_period': 0.32031691590835665, 'TXN_total_count': 0.27416354406095755, 'TXN_earn_count_per': 0.3475422282230056, 'TXN_redeem_count_per': 0.0, 'TXN_total_point': 0.0, 'TXN_earn_point_per': 0.0, 'TXN_redeem_point_per': 0.0, 'TXN_industry_count': 0.0, 'TXN_industry_per_retail': 0.0, 'TXN_industry_per_other': 0.0, 'TXN_industry_per_dingding': 0.0, 'TXN_industry_per_financial': 0.0, 'TXN_industry_per_digit': 0.0, 'TXN_industry_per_transportation': 0.0, 'TXN_industry_per_leisure': 0.0, 'TXN_industry_per_health': 0.0, 'TXN_industry_per_professional': 0.0, 'TXN_industry_per_food': 0.0, 'TXN_time_avg_period': 0.057977311807680156}
accuracy 0.5980200135322816
